In [ ]:
# # use erddapy to get some data
# from erddapy import ERDDAP
# import numpy as np
# 
# e = ERDDAP(
#     server="https://erddap.marine.usf.edu/erddap",
#     protocol="griddap",
# )
# 
# e.dataset_id = "jplMURSST41anom1day"
# print(f"variables in this dataset:\n\n{e.variables}")
# 
# # set bounds to reduce download size
# e.constraints = {
#     'time>=': '2003-06-07T09:00:00Z', 
#     'time<=': '2024-06-07T09:00:00Z', 
#     'time_step': 1,
#     'latitude>=': np.float64(lat - 1),
#     'latitude<=': np.float64(lat + 1),
#     'latitude_step': 1,
#     'longitude>=': np.float64(lon - 1),
#     'longitude<=': np.float64(lon + 1),
#     'longitude_step': 1
# }
# 
# Download as a NetCDF to disk
# nc_filename = e.download_file(".nc")  
# (under the hood this uses the GRiDDAP `.nc` URL)
# 
# ds = e.to_xarray()

In [19]:
%pip install dask pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 11.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
# sample all taxa occurrence points for ERDDAP data values:
# data has already been downloaded as .nc
# example:
# https://erddap.marine.usf.edu/erddap/griddap/cmems_salinity.graph?so%5B(2023-06-16)%5D%5B(0.494025)%5D%5B(-80.0):(90.0)%5D%5B(-180.0):(179.9167)%5D&.draw=surface&.vars=longitude%7Clatitude%7Cso&.colorBar=%7C%7C%7C%7C%7C&.bgColor=0xffccccff


import dask.dataframe as dd
occurrences = dd.read_csv("data/occurrences.csv", assume_missing=True)

import xarray as xr
ds = xr.open_dataset(
    "data/cmems_salinity_44d2_bc4b_70b2_U1749495233099.nc",
    chunks={"time": 1, "latitude": 500, "longitude": 500}
)


# build a Dask DataFrame of salinities
sal = ds["so"].sel(
    latitude=occurrences["decimalLatitude"],
    longitude=occurrences["decimalLongitude"],
    method="nearest"
)
# this is a dask Array; turn it into a dask Series
occurrences["salinity"] = sal.to_dask_dataframe(name="salinity")["salinity"]

# trigger computation in manageable chunks:
result = occurrences.compute()
print(result.head())

# for occurrence in occurrences.iterrows():
#     # adjust names if dims are named 'latitude'/'longitude' or 'lat'/'lon'
#     print(occurrence["decimalLatitude"])
#     val = ds["so"].sel(
#         latitude=occurrence["decimalLatitude"],
#         longitude=occurrence["decimalLongitude"],
#         method="nearest"
#     ).values

#     print(f"At ({occurrence["decimalLatitude"]}, {occurrence["decimalLongitude"]}), so = {val}")

# lat = 60
# lon = -60

# # Open file with xarray
# ds = xr.open_dataset("data/cmems_salinity_44d2_bc4b_70b2_U1749495233099.nc")


# # adjust names if dims are named 'latitude'/'longitude' or 'lat'/'lon'
# val = ds["so"].sel(
#     latitude=lat,
#     longitude=lon,
#     method="nearest"
# ).values

# print(f"At ({lat}, {lon}), so = {val}")

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------------------------+--------+----------+
| Column                   | Found  | Expected |
+--------------------------+--------+----------+
| accessRights             | object | float64  |
| associatedMedia          | object | float64  |
| associatedReferences     | object | float64  |
| collectionID             | object | float64  |
| continent                | object | float64  |
| countryCode              | object | float64  |
| county                   | object | float64  |
| dateIdentified           | object | float64  |
| disposition              | object | float64  |
| dynamicProperties        | object | float64  |
| eventRemarks             | object | float64  |
| eventType                | object | float64  |
| fieldNotes               | object | float64  |
| fieldNumber              | object | float64  |
| geodeticDatum            | object | float64  |
| georeferenceProtocol     | object | float64  